<a href="https://colab.research.google.com/github/vasantbala/ReAct-agent/blob/main/ReAct_Vanilla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.4 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('groq_api_key')

Instantiate LLM  and verify

In [5]:
from groq import Groq
client = Groq(api_key=os.environ['GROQ_API_KEY'])

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "Explain the importance of fast language models"}
    ],
    model="llama3-70b-8192",
    temperature=0
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's natural language processing (NLP) landscape, and their importance can be seen in several aspects:

1. **Real-time Applications**: Fast language models enable real-time applications such as chatbots, virtual assistants, and language translation systems to respond quickly and efficiently. This is particularly important in customer-facing applications where delayed responses can lead to frustration and a negative user experience.
2. **Low-Latency Requirements**: Many applications, such as voice assistants, require language models to process and respond to user input within a few hundred milliseconds. Fast language models can meet these low-latency requirements, ensuring a seamless user experience.
3. **Scalability**: Fast language models can handle large volumes of data and user requests, making them essential for large-scale NLP applications such as language translation, sentiment analysis, and text summarization.
4. **Energy Efficiency**: Fast

Setup agent

In [6]:
class Agent:
  def __init__(self, client: Groq, system: str = "") -> None:
    self.client = client
    self.system = system
    self.messages: list = []
    if self.system:
      self.messages.append({"role": "system", "content": self.system})

  def __call__(self, message=""):
    if message:
      self.messages.append({"role": "user", "content": message})
    result = self.execute()
    self.messages.append({"role": "assistant", "content": result})
    return result

  def execute(self):
    completion = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=self.messages,
        temperature=0
    )
    return completion.choices[0].message.content

Define prompt for our usecase. The prompt tells the LLM how to approach a problem. This prompt is a simple example and most definitely for demo purpose only

In [7]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.
Thought: I think I have found the answer
Action: Paris.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The capital of France is Paris

Example session

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth on Wikipedia
Action: wikipedia : mass of earth
PAUSE

You will be called again with this:

Observation: mass of earth is 1,1944×10e25

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

Define the Action Functions (tools)

In [9]:
import re
import httpx

def wikipedia(q):
  return httpx.get("https://en.wikipedia.org/w/api.php", params={
      "action": "query",
      "list": "search",
      "srsearch": q,
      "format": "json"
  }).json()["query"]["search"][0]["snippet"]
#
def calculate(operation: str) -> float:
  return eval(operation)
#

The next step is to define a function that uses an instantiation of the Agent. The loop function implements a loop that continues until there are no more actions (or we’ve reached the maximum number of iterations).

In [12]:
def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["calculate", "wikipedia"]

    next_prompt = query

    i = 0

    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            print(action)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break


Test the agent

In [13]:
loop(query="What is the current age of Donald Trump multiplied by 2?")

Thought: I need to find the birthdate of Donald Trump on Wikipedia.
Action: wikipedia: Donald Trump
PAUSE
[('wikipedia', 'Donald Trump')]
Observation: <span class="searchmatch">Donald</span> John <span class="searchmatch">Trump</span> (born June 14, 1946) is an American politician, media personality, and businessman who is the 47th president of the United States. A
Thought: I have found the birthdate of Donald Trump, now I need to calculate his current age.
Action: calculate: 2023 - 1946
PAUSE
[('calculate', '2023 - 1946')]
Observation: 77
Thought: I have calculated Donald Trump's current age, now I need to multiply it by 2.
Action: calculate: 77 * 2
PAUSE
[('calculate', '77 * 2')]
Observation: 154
Thought: I have calculated the result of multiplying Donald Trump's current age by 2.
Answer: The current age of Donald Trump multiplied by 2 is 154.


In [14]:
loop(query="What is the longest river in the World? What is the length of the river in km?")

Thought: I should look up the longest river in the world on Wikipedia.

Action: wikipedia: longest river
PAUSE
[('wikipedia', 'longest river')]
Observation: This is a list of the <span class="searchmatch">longest</span> <span class="searchmatch">rivers</span> on Earth. It includes <span class="searchmatch">river</span> systems over 1,000 kilometres (620 mi) in length. There are many factors, such as the
Thought: It seems like I have found a list of longest rivers, but I need to find the specific longest river and its length.

Action: wikipedia: Nile River
PAUSE
[('wikipedia', 'Nile River')]
Observation: The <span class="searchmatch">Nile</span> (also known as the <span class="searchmatch">Nile</span> <span class="searchmatch">River</span> or <span class="searchmatch">River</span> <span class="searchmatch">Nile</span>) is a major north-flowing <span class="searchmatch">river</span> in northeastern Africa. It flows into the Mediterranean Sea. The Nile
Thought: I think I have found the lo